In [1]:
import collections
import re
from d2l import torch as d2l

F:\programData\conda\env\ChatGLM2-6B\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
F:\programData\conda\env\ChatGLM2-6B\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
F:\programData\conda\env\ChatGLM2-6B\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [4]:
#8.2.1. 读取数据集
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')
def read_time_machine():
    with open(d2l.download('time_machine'),'r') as f:
        lines = f.readlines()
    return [re.sub('^A-Za-z'+',',' ',line).strip().lower() for line in lines]

lines = read_time_machine()
print(f"# 文本总行数：{len(lines)}")
print(lines[0])
print(lines[10])

# 文本总行数：3221
the time machine, by h. g. wells [1898]
twinkled, and his usually pale face was flushed and animated. the


In [5]:
#8.2.2. 词元化
def tokenize(lines,token='word'):
    if token == 'word':
        return [line.split() for line in lines]
    elif token == "char":
        return [list(line) for line in lines]
    else:
        print("错误：未知词元类型：" + token)
tokens = tokenize(lines)
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine,', 'by', 'h.', 'g.', 'wells', '[1898]']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', '(for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him)']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us.', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled,', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated.', 'the']


In [6]:
#8.2.3. 词表
class Vocab:
    def __init__(self,tokens=None,min_freq=0,reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []

        counter = count_corpus(tokens)
        self._token_freqs = sorted(counter.items(),key=lambda x:x[1],reverse=True)
        self.idx_to_token = ['<unk>'] + reserved_tokens
        self.token_to_idx = {token:idx for idx,token in enumerate(self.idx_to_token)}

        for token,freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token]  = len(self.idx_to_token) -1

    def __len__(self):
        return len(self.idx_to_token)
    def __getitem__(self,tokens):
        if not isinstance(tokens,(list,tuple)):
            return self.token_to_idx.get(tokens,self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self,indices):
        if not isinstance(indices,(list,tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]


    @property
    def unk(self):
        return 0
    #property
    def token_freqs(self):
        return self._token_freqs

def count_corpus(tokens):
    if len(tokens) == 0 or isinstance(tokens[0],list):
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

In [8]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('and', 2), ('i', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('my', 9)]


In [9]:
for i in [0,10]:
    print("文本：",tokens[i])
    print("索引：",vocab[tokens[i]])

文本： ['the', 'time', 'machine,', 'by', 'h.', 'g.', 'wells', '[1898]']
索引： [1, 18, 197, 37, 2413, 2414, 755, 2415]
文本： ['twinkled,', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated.', 'the']
索引： [2419, 2, 22, 996, 442, 154, 7, 1410, 2, 1411, 1]


In [14]:
#8.2.4. 整合所有功能
def load_corpus_time_machine(max_tokens=-1):
    lines = read_time_machine()
    tokens = tokenize(lines,'char')
    vocab = Vocab(tokens)

    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus,vocab

corpus,vocab = load_corpus_time_machine()
len(corpus),len(vocab)

(175758, 45)